# DataLab Getting Started in Scala

The Bigstep DataLab is a open data exploration service that offers data science, analytics and technology experimentation, built on our SparkArray, DataLake and on our highly flexible and high performance bare-metal infrastructure.

This tutorial assumes some programming experience.

## Uploading Data

A private datalake (HDFS service) is used to store the data that the SparkArray uses. To upload data to an HDFS cluster one would typically:
1. download the hadoop binaries (2.7.x) from a mirror like [here](http://apache.claz.org/hadoop/common/hadoop-2.7.3/hadoop-2.7.3.tar.gz)  - rather large  (240mb)
2. unarchive 
3. execute commands like "-ls".

```
/hadoop-2.7.3/bin/hdfs dfs -ls hdfs://headnodes-8885.cluster-8885.us-private-datalake.7.bigstep.io/
16/09/26 17:18:47 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 3 items
drwxrwxrwx   - hdfs supergroup          0 2016-09-22 13:12 hdfs://headnodes-8885.cluster-8885.us-private-datalake.7.bigstep.io/baseball
drwxrwxrwt   - hdfs supergroup          0 2016-09-22 12:08 hdfs://headnodes-8885.cluster-8885.us-private-datalake.7.bigstep.io/tmp
drwxr-xr-x   - hdfs supergroup          0 2016-09-22 12:08 hdfs://headnodes-8885.cluster-8885.us-private-datalake.7.bigstep.io/user
```


You can also execute the same commands on the master container:

In [ ]:
/* import a SparkSession */
import org.apache.spark.sql.SparkSession

/* Create a SparkSession that is conencted to the current Spark Master. Retrieve the SparkMaster URL from the
   Spark tab in the Bigstep Control Center and replace the value in the line below: */
val sparkSession = SparkSession.builder.master("spark://container-cluster-2367:7077").appName("my-spark-app").getOrCreate()

/* Allow the use of shell operations */
import sys.process._

"wget http://www.exploredata.net/ftp/MLB2008.csv" !

In [ ]:
/* Copy the downloaded file to Bigstep DataLake, using the relative path of the user's DataLake home directory */
"hdfs dfs -put MLB2008.csv /" !

## Initialize Spark Context

For all Spark functions to be available, a Spark context is initialized by default in the current notebook.

In [ ]:
sc

## RDDs

An Resilient Distributed Dataset is an array that is spread across multiple servers. It allows the programmer to abstract away the complexity of transforming large volumes of distributed data.

In [ ]:
"wget http://seanlahman.com/files/database/baseballdatabank-master_2016-03-02.zip" !

"apt-get install -y unzip" !
"unzip baseballdatabank-master_2016-03-02.zip" !
"rm -rf baseballdatabank-master_2016-03-02.zip" !

"hdfs dfs -put baseballdatabank-master/core/AllstarFull.csv /" !

In [ ]:
val textFile = sc.textFile("/AllstarFull.csv")

In [ ]:
textFile.count()

In [ ]:
textFile.first()

In [ ]:
val linesWithRuth = textFile.filter( line => line.contains("ruth"))

In [ ]:
linesWithRuth.count()

In [ ]:
linesWithRuth.collect()

In [ ]:
linesWithRuth.saveAsTextFile("/lines_with_ruth-file")

# DataFrames and SparkSQL

A SparkDataFrame can also be registered as a temporary view in Spark SQL and that allows you to run SQL queries over its data. The sql function enables applications to run SQL queries programmatically and returns the result as a SparkDataFrame.

Spark 2.1.0 has a built-in CSV reader:

In [ ]:
"hdfs dfs -chmod 740 /AllstarFull.csv" !

In [ ]:
val allstar = sparkSession.read.option("header", "true").csv("/AllstarFull.csv")

In [ ]:
allstar.take(10)

In [ ]:
allstar

In [ ]:
allstar.show()

In [ ]:
/* register this table as a "table" within the sql context. */
allstar.createOrReplaceTempView("allstar")

/* SQL can be run over DataFrames that have been registered as a table. */
val player = spark.sql("SELECT * FROM allstar WHERE playerID like '%ruth%' and yearID<1935")

In [ ]:
player.show()

## ParquetFiles

Parquet files are typically much faster and take up less space than CSVs and the DataLake supports them as well. As Spark is a clustering system the parquet files are composed out of many fragments generated by the workres independently. The collection of files is operated as a single big table by SparkSQL.

To write the dataframe:

In [ ]:
import java.util.Calendar

val now = Calendar.getInstance().getTimeInMillis()

val path="/allstar-"+now.toString()+".parquet"
player.write.save(path)


Read the dataframe:

In [ ]:
val dfParquet = spark.read.parquet(path)
dfParquet.createOrReplaceTempView("player")
spark.sql("SELECT playerID,YearID FROM player").show()

## Resources 

[Apache Spark 2.1.0 Programming Guide](http://spark.apache.org/docs/latest/sql-programming-guide.html)